<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Откройте-и-изучите-файл" data-toc-modified-id="Откройте-и-изучите-файл-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Откройте и изучите файл</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Разбейте-данные-на-выборки" data-toc-modified-id="Разбейте-данные-на-выборки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разбейте данные на выборки</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Исследуйте-модели" data-toc-modified-id="Исследуйте-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследуйте модели</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Решающее-дерево" data-toc-modified-id="Решающее-дерево-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Решающее дерево</a></span></li><li><span><a href="#Рандомный-лес" data-toc-modified-id="Рандомный-лес-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Рандомный лес</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Проверьте-модель-на-тестовой-выборке" data-toc-modified-id="Проверьте-модель-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверьте модель на тестовой выборке</a></span></li><li><span><a href="#(бонус)-Проверьте-модели-на-адекватность" data-toc-modified-id="(бонус)-Проверьте-модели-на-адекватность-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>(бонус) Проверьте модели на адекватность</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
* `сalls` — количество звонков,
* `minutes` — суммарная длительность звонков в минутах,
* `messages` — количество sms-сообщений,
* `mb_used` — израсходованный интернет-трафик в Мб,
* `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Обзор данных

In [38]:
import pandas as pd
import numpy as np
import time
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

In [20]:
try:
    data = pd.read_csv('/datasets/users_behavior.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')

In [21]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [23]:
data.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


### Вывод

Пропусков нет, все типы данных в порядке, значения тоже выглядят нормально. Предобработка и правда не нужна.

## Разбейте данные на выборки

Целевой признак это тариф, выделим его в одну переменную, а все остальные признаки в дргую.

In [25]:
features = data.drop(['is_ultra'], axis=1)
target = data['is_ultra']
print(features.shape)
target.shape

(3214, 4)


(3214,)

In [26]:
target.value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

Доля классов не равна. Но треть это достаточно много, обучить модель можно.

Датасет делим на три части: тренировочная, валидационная и тестовая в соотношении 3:1:1.  
Сначачла выделим 60% в тренировочную часть, потом оставшиеся 40% поделим поплам.

In [43]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.4, random_state=123)

features_test, features_valid, target_test, target_valid  = train_test_split(features_test, target_test, test_size=0.5, random_state=123) 

print('Тренировочные данные: ', features_train.shape, target_train.shape)
print('Валидационные данные: ', features_valid.shape, target_valid.shape)
print('Тестовые данные: ', features_test.shape, target_test.shape)

Тренировочные данные:  (1928, 4) (1928,)
Валидационные данные:  (643, 4) (643,)
Тестовые данные:  (643, 4) (643,)


### Вывод

Данные разделены на три выборки: тренировочная, валидационная и тестовая в соотношении 3:1:1.  
На тренировочных данных модели будут обучаться, по валидационным данным проведём предварительное сравнение точности, а по тестовым данным финальное сравнение точности.

## Исследуйте модели

Задачу классификации мы умеем решать тремя методами: решающее дерево, рандомный лес и логистическая регрессия.   
Какие гиперпараметры у этих моделей я буду менять:  
* дерево: глубина дерева;
* рандомный лес: количество деревьев и их глубина;
* логистическая регрессия: алгоритм и количество итераций;

### Логистическая регрессия

In [28]:
solvers=['liblinear', 'lbfgs']
best_linear_model = None
best_linear_result = 0
best_linear_solver = ''
best_linear_iter = 0

for solver in solvers:
    for i in range(100,1001,100):        
        linear = LogisticRegression(solver=solver, max_iter=i, random_state=123)
        linear.fit(features_train, target_train)
        linear_prediction = linear.predict(features_valid)
        result = accuracy_score(linear_prediction, target_valid)
        if result > best_linear_result:
            best_linear_model = linear
            best_linear_result = result
            best_linear_solver = solver
            best_linear_iter = i

print('Лчший результат:\nТочность:', best_linear_result, 
      '\nАлгоритм:', best_linear_solver, 
      '\nИтераций', best_linear_iter)

Лчший результат:
Точность: 0.7309486780715396 
Алгоритм: lbfgs 
Итераций 100


Логистическая регрессия не смогла дать нам желаемое значение точности в 0.75.

### Решающее дерево

In [29]:
best_tree_model = None
best_tree_result = 0
best_tree_depth = 0

for depth in range(1,11):
    tree = DecisionTreeClassifier(max_depth=depth, random_state=123)
    tree.fit(features_train, target_train)
    tree_prediction = tree.predict(features_valid)
    result = accuracy_score(tree_prediction, target_valid)
    if result > best_tree_result:
            best_tree_model = tree
            best_tree_result = result
            best_tree_depth = depth
            
print('Лчший результат:\nТочность:', best_tree_result, '\nГлубина: ', best_tree_depth)

Лчший результат:
Точность: 0.7962674961119751 
Глубина:  3


Решающее дерево получило точность почти 80% на валидационных данных. Можно проверять на тестовых.

### Рандомный лес

In [35]:
start_time = time.time()

best_forest_model = None
best_forest_result = 0
best_forest_depth = 0
best_forest_est = 0

for est in range(10, 51, 10):
    for depth in range(1,11):
        forest = RandomForestClassifier(n_estimators=est, max_depth=depth, random_state=123)
        forest.fit(features_train, target_train)
        forest_prediction = forest.predict(features_valid)
        result = accuracy_score(forest_prediction, target_valid)
        if result > best_forest_result:
                best_forest_model = forest
                best_forest_result = result
                best_forest_depth = depth
                best_forest_est = est
            
print('Лчший результат:\nТочность:', best_forest_result,
      '\nЧисло деревьев',best_forest_est, 
      '\nГлубина: ', best_forest_depth)
print("--- %s seconds ---" % (time.time() - start_time))

Лчший результат:
Точность: 0.8102643856920684 
Число деревьев 10 
Глубина:  6
--- 3.746145486831665 seconds ---


Рандомный лес показал наилучший результат в 81% точности на валидационной выборке.   
Однако перебор вариантов занял больше времени. На моём ПК (довольно слабом) задержка в выполнении уже заметна и составляет 4 секунды.   
Так как наилучший результат показал почти самый маленький лес, то его доля в вычислениях небольшая. Однако всё таки значительно больше, чем для дерева.

### Вывод

Логистическая регрессия не справилась даже с поставленной целью в 75% точности. Лучший результат: 73%.  
Решающее дерево показало хороший результат. При глубине равной 3, точность составила 79.6% точности.  
Рандомный лес показал наилучший результат: 81% при 10 деревьях и глубине 6. Однако этот вариант более медленный.  

На тестовых данных проверим лучшие дерево и лес.

## Проверьте модель на тестовой выборке

In [32]:
best_forest_prediction = best_forest_model.predict(features_test)
best_tree_prediction = best_tree_model.predict(features_test)

test_forest_result = accuracy_score(best_forest_prediction, target_test)
test_tree_result = accuracy_score(best_tree_prediction, target_test)

print('Точность леса на тестовых данных:', test_forest_result)
print('Точность дерева на тестовых данных:', test_tree_result)

Точность леса на тестовых данных: 0.8164852255054432
Точность дерева на тестовых данных: 0.80248833592535


Ожидаемый результат: лес лучше дерева: 81.6% против 80.2%.   

In [33]:
features_test = pd.concat([features_valid, features_test], ignore_index=True)
target_test = pd.concat([target_valid, target_test], ignore_index=True)

In [34]:
best_forest_prediction = best_forest_model.predict(features_test)
best_tree_prediction = best_tree_model.predict(features_test)

test_forest_result = accuracy_score(best_forest_prediction, target_test)
test_tree_result = accuracy_score(best_tree_prediction, target_test)

print('Точность леса на тестовых данных:', test_forest_result)
print('Точность дерева на тестовых данных:', test_tree_result)

Точность леса на тестовых данных: 0.8133748055987559
Точность дерева на тестовых данных: 0.7993779160186625


## (бонус) Проверьте модели на адекватность

Адекватность классификационной задачи по-моему совпадает с точностью.  
Если мы случайным образом расставим классы, то точность будет примерно 50%.   
У нашей модели точность выше 80%, что делает её лучше случайной, а значит адекватной.

Однако можно учесть распределение классов в нашем наборе и расставлять 0 и 1 не 50/50, а с соответствующей вероятностью.  
Проверку будем делать на тестовых данных, посмотрим распределение на ней.

In [14]:
target_test.value_counts()

0    449
1    194
Name: is_ultra, dtype: int64

Создадим массив из 449 нулей и 194 единиц, а затем перемешаем его и посчитаем точность.

In [15]:
zeros = np.zeros(449)
ones = np.ones(194)
random = np.concatenate((zeros, ones))
np.random.shuffle(random)
accuracy_score(random, target_test)

0.5987558320373251

Точность такого метода составила примерно 58%. Наши модели справились лучше.

In [39]:
dummy = DummyClassifier(strategy='most_frequent').fit(features_train, target_train)
dummy_prediction = dummy.predict(features_test)
accuracy_score(target_test, dummy_prediction)

0.6959564541213064

Эту проверку наша модель тоже проходит.

## Вывод

Предоставленные данные выглядят хорошо и не нуждаются в предобработке.  

Данные разделены на три выборки: тренировочная, валидационная и тестовая в соотношении 3:1:1.
На тренировочных данных модели обучались, по валидационным данным было проведено предварительное сравнение точности, а по тестовым данным финальное сравнение точности.

Задача решалась тремя методами: решающее дерево, рандомный лес и логистическая регрессия.   
Логистическая регрессия не справилась даже с поставленной целью в 75% точности. Лучший результат: 73%.  
Решающее дерево показало хороший результат. При глубине равной 3, точность составила 79.6% точности.  
Рандомный лес показал наилучший результат: 81% при 10 деревьях и глубине 6. Однако этот вариант более медленный.  
На тестовых данных проверялись дерево и лес, показавшие наилучший результат на валидационных данных.  

На тестовых данных лес показал точность 81.6% против 80.2% у дерева.   
Разница составляет всего 1.4%, так что окончательный выбор состоит в необходимости совершать вычисления быстро. Для быстрых вычислений следует выбрать дерево, для медленных - лес.